In [2]:
from bs4 import SoupStrainer
from langchain import hub 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [1]:
# 환경변수 불러오기
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
# Load Document 
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=SoupStrainer(
            class_=("post-header", "post-content")
        )
    )
)

docs = loader.load()

print(docs)

[Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final re

In [12]:
# Text Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits), splits)

66 [Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\

In [14]:
# Embed
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [15]:
# Prompt 
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [17]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [18]:
# Question 
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents to plan and execute tasks more efficiently by dividing them into manageable components. Task decomposition can be achieved through various methods such as prompting with specific instructions or utilizing human inputs.'

In [7]:
import requests 
from bs4 import BeautifulSoup, SoupStrainer

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

parse_only = SoupStrainer(class_=["post-header","post-content"])

html = requests.get(url)
if html.status_code == 200:
    soup = BeautifulSoup(html.text, parse_only=parse_only)

print(soup.prettify())

<!DOCTYPE html>
<header class="post-header">
 <h1 class="post-title">
  LLM Powered Autonomous Agents
 </h1>
 <div class="post-meta">
  Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng
 </div>
</header>
<div class="post-content">
 <p>
  Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as
  <a href="https://github.com/Significant-Gravitas/Auto-GPT">
   AutoGPT
  </a>
  ,
  <a href="https://github.com/AntonOsika/gpt-engineer">
   GPT-Engineer
  </a>
  and
  <a href="https://github.com/yoheinakajima/babyagi">
   BabyAGI
  </a>
  , serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
 </p>
 <h1 id="agent-system-overview">
  Agent System Overview
  <a aria-hidden="true" class="anchor" hidden="" href="#agent-system-overview">
   #
  </a>
 </h1>
 <p>
  In 

In [20]:
import requests 
from bs4 import BeautifulSoup, SoupStrainer

url = "https://n.news.naver.com/mnews/article/421/0007578326"

parse_only = SoupStrainer(class_=["media_end_head_headline","go_trans _article_content"])

html = requests.get(url)
if html.status_code == 200:
    soup = BeautifulSoup(html.text, parse_only=parse_only)

print(soup.prettify())

<!DOCTYPE html>
<h2 class="media_end_head_headline" id="title_area">
 <span>
  "AI 학습·공익서비스 개발" 빅데이터…가명처리로 안전하게 사용
 </span>
</h2>
<article class="go_trans _article_content" id="dic_area">
 <strong class="media_end_summary">
  전세계 데이터 75% '개인정보'…"규제 막혀 버려져선 안돼"
  <br/>
  데이터 '마스킹·대체' 가명정보…"기관간 결합해 시너지"
 </strong>
 <span class="end_photo_org">
  <div class="nbd_im_w _LAZY_LOADING_WRAP">
   <div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
    <img alt="가명 정보끼리 결합하면 데이터에 근거한 공공서비스 개발도 가능하다. 복지 사각지대 위험군을 추적하는 데 쓰일 수 있다.(한국인터넷진흥원 제공)" class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" data-src="https://imgnews.pstatic.net/image/421/2024/06/03/0007578326_001_20240603110035335.jpg?type=w647" id="img1" style="display: none;"/>
   </div>
  </div>
  <em class="img_desc">
   가명 정보끼리 결합하면 데이터에 근거한 공공서비스 개발도 가능하다. 복지 사각지대 위험군을 추적하는 데 쓰일 수 있다.(한국인터넷진흥원 제공)
  </em>
 </span>
 <br/>
 <br/>
 (서울=뉴스1) 윤주영 기자 = 인공지능(AI)을 고도화하는 학습 데이터엔 개인 민감정보가 들어있어 자칫 개인정보보호법을 침해할 소지가 있다. 정부는 이를 막으면서도 기술 고도화 토대를 마련하고자 개

In [62]:
naver_client_id = "VzPpKwAPQWA3MaI1kZpL"
naver_secret_key = "MqknyFulXf"

import os 
from dotenv import load_dotenv
from urllib.parse import urlencode
from urllib.request import Request, urlopen

load_dotenv()

keyword = "의대생 살인사건"
base_url = "https://openapi.naver.com/v1/search/news?"
params = {
    "query": keyword,
    "display": 10,
    "sort": "sim"
}

url = base_url + urlencode(params)
headers = {
    "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
    "X-Naver-Client-Secret": os.getenv("NAVER_SECRET_KEY")
}

request = Request(url, headers=headers)
response = urlopen(request)

print(response.read().decode("utf-8"))

{
	"lastBuildDate":"Mon, 03 Jun 2024 17:35:23 +0900",
	"total":721,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"'<b>의대생 살인<\/b>' 한달도 안돼 또…'헤어지자' 한마디에 <b>살인<\/b>자 돌변 왜?",
			"originallink":"https:\/\/www.news1.kr\/articles\/5435018",
			"link":"https:\/\/n.news.naver.com\/mnews\/article\/421\/0007577585?sid=102",
			"description":"지난달 2일 발생한 '<b>의대생 살인<\/b>' <b>사건<\/b>이 발생한 지 한 달도 되지 않아 발생한 '교제 <b>살인<\/b>' <b>사건<\/b>이다. 경찰은 박 씨가 이별을 통보받자 보복 목적으로 범행했을 가능성에 무게를 두고 수사하고 있다. 박 씨는 우발적인... ",
			"pubDate":"Mon, 03 Jun 2024 06:30:00 +0900"
		},
		{
			"title":"또 교제<b>살인<\/b>… “사람 죽였다” 30대男 자수 후 숨져",
			"originallink":"https:\/\/www.donga.com\/news\/Society\/article\/all\/20240602\/125236250\/1",
			"link":"https:\/\/n.news.naver.com\/mnews\/article\/020\/0003568235?sid=102",
			"description":"최근 ‘<b>의대생 살인 사건<\/b>’ 등 교제하던 이성을 살해하는 <b>사건<\/b>이 이어지며 ‘교제폭력’ 대책을 마련해야 한다는 목소리가 높아진다. 2일 경남 창녕경찰서에 따르면 지난달 28일 낮 12시경 “내가 사람을 죽였다.... ",
			"pubDate":"Sun, 02 Jun 2024 17:39:00 +0900"
		},
		{


In [81]:
import json 

def _search_naver_news(keyword):
    base_url = "https://openapi.naver.com/v1/search/news?"
    params = urlencode({
        "query": keyword,
        "display": 10,
        "sort": "sim"
    })

    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_SECRET_KEY")
    }

    request = Request(base_url + params, headers=headers)
    response = urlopen(request).read().decode("utf-8")
    json_data = json.loads(response)

    return json_data["items"]

test = _search_naver_news("의대생 살인사건")
test
    

[{'title': "'<b>의대생 살인</b>' 한달도 안돼 또…'헤어지자' 한마디에 <b>살인</b>자 돌변 왜?",
  'originallink': 'https://www.news1.kr/articles/5435018',
  'link': 'https://n.news.naver.com/mnews/article/421/0007577585?sid=102',
  'description': "지난달 2일 발생한 '<b>의대생 살인</b>' <b>사건</b>이 발생한 지 한 달도 되지 않아 발생한 '교제 <b>살인</b>' <b>사건</b>이다. 경찰은 박 씨가 이별을 통보받자 보복 목적으로 범행했을 가능성에 무게를 두고 수사하고 있다. 박 씨는 우발적인... ",
  'pubDate': 'Mon, 03 Jun 2024 06:30:00 +0900'},
 {'title': '또 교제<b>살인</b>… “사람 죽였다” 30대男 자수 후 숨져',
  'originallink': 'https://www.donga.com/news/Society/article/all/20240602/125236250/1',
  'link': 'https://n.news.naver.com/mnews/article/020/0003568235?sid=102',
  'description': '최근 ‘<b>의대생 살인 사건</b>’ 등 교제하던 이성을 살해하는 <b>사건</b>이 이어지며 ‘교제폭력’ 대책을 마련해야 한다는 목소리가 높아진다. 2일 경남 창녕경찰서에 따르면 지난달 28일 낮 12시경 “내가 사람을 죽였다.... ',
  'pubDate': 'Sun, 02 Jun 2024 17:39:00 +0900'},
 {'title': '[만물상] 60대 데이트 <b>살인</b>',
  'originallink': 'https://www.chosun.com/opinion/manmulsang/2024/06/02/VDNW7HM6BFCNHGTN46FULOUY4Q/?utm_source=naver&utm

In [24]:
import os
import json
from dotenv import load_dotenv
from urllib.parse import urlencode
from urllib.request import Request, urlopen

def _search_naver_news(keyword):
    # url 만들기
    base_url = "https://openapi.naver.com/v1/search/news?"
    params = {
        "query": keyword,
        "display": 20,
        "sort": "sim"
    }
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_SECRET_KEY")
    }

    url = base_url + urlencode(params)

    # 네이버 Open API 요청하기
    request = Request(url, headers=headers)
    response = urlopen(request) 
    
    # dictionary로 변환하기
    response_str = response.read().decode("utf-8")
    str2json = json.loads(response_str)
    
    # url 리스트 만들기
    url_list = [url["link"] for url in str2json["items"] if "naver" in url["link"]]

    return url_list


keyword = "서울대 의대생 살인"
news = _search_naver_news(keyword)
print(len(news))
news

14


['https://n.news.naver.com/mnews/article/023/0003832910?sid=102',
 'https://n.news.naver.com/mnews/article/016/0002305823?sid=102',
 'https://n.news.naver.com/mnews/article/015/0004981811?sid=102',
 'https://n.news.naver.com/mnews/article/029/0002872434?sid=102',
 'https://n.news.naver.com/mnews/article/011/0004338290?sid=102',
 'https://n.news.naver.com/mnews/article/008/0005039360?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005744949?sid=102',
 'https://n.news.naver.com/mnews/article/053/0000040875?sid=100',
 'https://n.news.naver.com/mnews/article/277/0005368392?sid=102',
 'https://n.news.naver.com/mnews/article/023/0003835058?sid=110',
 'https://n.news.naver.com/mnews/article/079/0003895834?sid=102',
 'https://n.news.naver.com/mnews/article/422/0000661880?sid=102',
 'https://n.news.naver.com/mnews/article/032/0003295367?sid=102',
 'https://n.news.naver.com/mnews/article/056/0011717282?sid=004']

In [ ]:
from bs4 import SoupStrainer
from langchain import hub 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [27]:
# Load Document 
loader = WebBaseLoader(
    web_paths=tuple(news),
    bs_kwargs=dict(
        parse_only=SoupStrainer(
            class_=["media_end_head_headline","go_trans _article_content"]
        )
    )
)

docs = loader.load()
print(len(docs))
print(docs)

14
[Document(page_content='“프로필 사진이 의대생 여친” 피해자 신상까지 터는 네티즌들\n\n\n\n\n일러스트=이철원\t\t\t\t\t\t\t\t\t\t강남역 인근 건물 옥상에서 여자친구를 흉기로 살해한 20대 남성이 ‘수능 만점 의대생’으로 알려지면서 그의 신상과 사진 등이 온라인에서 급속도로 퍼지고 있다.8일 경찰에 따르면 살인 혐의로 긴급 체포된 A(25)씨는 지난 6일 오후 5시쯤 서울 서초구 서초동 지하철 2호선 강남역 근처 건물 옥상에서 동갑내기 여자친구에게 여러 차례 흉기를 휘둘러 숨지게 한 혐의를 받는다.A씨는 수능 만점을 받고, 서울의 한 명문대에 재학 중인 의대생이다. 이후 네티즌들은 A씨가 수능을 치렀을 것으로 추정되는 연도의 수능 만점자를 다룬 기사 등을 통해 그의 신상을 특정했다.경기도의 한 일반계 고등학교를 졸업한 A씨는 당시 여러 매체와 인터뷰를 했다. 외과 의사를 꿈꾸던 A씨는 수능 선택 과목으로 인해 서울대 지원이 불가능해 다른 최상위권 의대를 택한 것으로 전해졌다. 서울대 의대 정시에 지원하기 위해서는 과학탐구 영역에서 Ⅰ+Ⅱ, Ⅱ+Ⅱ 조합을 만족해야 한다.\n\n\n\n7일 온라인 커뮤니티에 올라온 여자친구 살인 혐의 의대생에 관한 신상 글. /온라인 커뮤니티\t\t\t\t\t\t\t\t\t\t네티즌들은 A씨의 소셜미디어 계정과 피해자의 소셜미디어 계정도 찾아냈다. 온라인 커뮤니티에는 “SNS 프로필 사진이 여자친구와 찍은 사진이더라” “여자친구가 마지막으로 올린 글이 부모님과 함께한 사진이던데, 어버이날 전날에 이게 무슨 일이냐” 등의 글이 다수 올라왔다.일각에서는 무분별한 ‘신상 털기’에 대한 우려의 목소리도 나왔다. 네티즌들은 “가족과 친구들은 무슨 죄냐” “피해자 신상까지 터는 이유는 뭐냐” “애꿎은 사람까지 피해를 볼까 걱정된다” 등의 반응을 보였다.그동안 경찰은 신상공개심의위원회를 통해 강력범의 신상정보를 공개하더라도, 공개된 정보 외에 가족이나 지인에 관한 신상을 공개하거나 온라인에 모욕적인

In [28]:
# Text Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits), splits)

51 [Document(page_content='“프로필 사진이 의대생 여친” 피해자 신상까지 터는 네티즌들\n\n\n\n\n일러스트=이철원\t\t\t\t\t\t\t\t\t\t강남역 인근 건물 옥상에서 여자친구를 흉기로 살해한 20대 남성이 ‘수능 만점 의대생’으로 알려지면서 그의 신상과 사진 등이 온라인에서 급속도로 퍼지고 있다.8일 경찰에 따르면 살인 혐의로 긴급 체포된 A(25)씨는 지난 6일 오후 5시쯤 서울 서초구 서초동 지하철 2호선 강남역 근처 건물 옥상에서 동갑내기 여자친구에게 여러 차례 흉기를 휘둘러 숨지게 한 혐의를 받는다.A씨는 수능 만점을 받고, 서울의 한 명문대에 재학 중인 의대생이다. 이후 네티즌들은 A씨가 수능을 치렀을 것으로 추정되는 연도의 수능 만점자를 다룬 기사 등을 통해 그의 신상을 특정했다.경기도의 한 일반계 고등학교를 졸업한 A씨는 당시 여러 매체와 인터뷰를 했다. 외과 의사를 꿈꾸던 A씨는 수능 선택 과목으로 인해 서울대 지원이 불가능해 다른 최상위권 의대를 택한 것으로 전해졌다. 서울대 의대 정시에 지원하기 위해서는 과학탐구 영역에서 Ⅰ+Ⅱ, Ⅱ+Ⅱ 조합을 만족해야 한다.', metadata={'source': 'https://n.news.naver.com/mnews/article/023/0003832910?sid=102'}), Document(page_content='7일 온라인 커뮤니티에 올라온 여자친구 살인 혐의 의대생에 관한 신상 글. /온라인 커뮤니티\t\t\t\t\t\t\t\t\t\t네티즌들은 A씨의 소셜미디어 계정과 피해자의 소셜미디어 계정도 찾아냈다. 온라인 커뮤니티에는 “SNS 프로필 사진이 여자친구와 찍은 사진이더라” “여자친구가 마지막으로 올린 글이 부모님과 함께한 사진이던데, 어버이날 전날에 이게 무슨 일이냐” 등의 글이 다수 올라왔다.일각에서는 무분별한 ‘신상 털기’에 대한 우려의 목소리도 나왔다. 네티즌들은 “가족과 친구들은 무슨 죄냐” “피해자 신상까지 터는 이유는 뭐냐” “애꿎은 사람까지 피

In [29]:
# Embed
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [30]:
# Prompt 
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [31]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

In [34]:
# Question 
rag_chain.invoke("참외는 얼마래?")

'저는 참외의 가격에 대한 정보를 알 수 없습니다.'

In [35]:
from langchain import hub 

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [36]:
template = """
# INSTRUCTION
- ARTICLE을 FORMAT에 맞춰 5줄로 요약하세요.
- 요약은 bullet form으로 작성하고 온점(.)으로 종결하세요.
- 태그는 ARTICLE을 대표하는 키워드를 의미합니다.
- 태그는 최대 5개이며, 키워드 앞에 '\#'를 붙이세요.


# ARTICLE: {article}

# FORMAT:
🔖 태그1, 태그2, ...

📌 요점 정리
- 요약1
- 요약2
- ...

# EXAMPLES
1. 
🔖 #양자점레이저, #ETRI, #광통신부품연구실

📌 요점 정리

- 양자점 레이저 대량 생산 기술 개발로 생산단가 1/6 수준으로 낮아질 전망.
- ETRI가 MOCVD를 이용해 인듐비소/갈륨비소 양자점 레이저 다이오드 개발 성공.
- 기존 MBE 방식보다 밀도가 높고 균일한 양자점 제조 기술 개발.
- 통신용 반도체 레이저 제조비용을 1/6 이하로 낮출 수 있을 것으로 기대.
- 연구 성과를 통해 해외시장 점유율 증가 및 국내 광통신 부품 산업 성장 기대.

2. 
🔖 #SK텔레콤, #AI, #소비자만족도

📌 요점 정리

- SK텔레콤, NCSI 이동전화 서비스 부문 27년 연속 1위 차지.
- AI 기술을 활용한 서비스 혁신으로 소비자만족도 80점 돌파.
- 에이닷 AI비서와 스페셜T 프로그램 등으로 성과 이룸.
- 매장 및 고객센터에서 AI 기술 활용해 소비자 편의 증진.
- 가족 로밍, 0 청년 요금제 등 다양한 혜택 제공으로 고객 만족도 높임.
"""

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

ChatPromptTemplate(input_variables=['article'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['article'], template="# INSTRUCTION\n- 당신은 뉴스 정보를 전달하는 기자입니다.\n- ARTICLE을 FORMAT에 맞춰 5줄로 요약하세요.\n- 요약은 bullet form으로 작성하고 온점(.)으로 종결하세요.\n- 태그는 ARTICLE을 대표하는 키워드를 의미합니다.\n- 태그는 최대 5개이며, 키워드 앞에 '\\#'를 붙이세요.\n\n\n# ARTICLE: {article}\n\n# FORMAT:\n🔖 태그1, 태그2, ...\n\n📌 요점 정리\n- 요약1\n- 요약2\n- ...\n\n# EXAMPLES\n1. \n🔖 #양자점레이저, #ETRI, #광통신부품연구실\n\n📌 요점 정리\n\n- 양자점 레이저 대량 생산 기술 개발로 생산단가 1/6 수준으로 낮아질 전망.\n- ETRI가 MOCVD를 이용해 인듐비소/갈륨비소 양자점 레이저 다이오드 개발 성공.\n- 기존 MBE 방식보다 밀도가 높고 균일한 양자점 제조 기술 개발.\n- 통신용 반도체 레이저 제조비용을 1/6 이하로 낮출 수 있을 것으로 기대.\n- 연구 성과를 통해 해외시장 점유율 증가 및 국내 광통신 부품 산업 성장 기대.\n\n2. \n🔖 #SK텔레콤, #AI, #소비자만족도\n\n📌 요점 정리\n\n- SK텔레콤, NCSI 이동전화 서비스 부문 27년 연속 1위 차지.\n- AI 기술을 활용한 서비스 혁신으로 소비자만족도 80점 돌파.\n- 에이닷 AI비서와 스페셜T 프로그램 등으로 성과 이룸.\n- 매장 및 고객센터에서 AI 기술 활용해 소비자 편의 증진.\n- 가족 로밍, 0 청년 요금제 등 다양한 혜택 제공으로 고객 만족도 높임.\n"))])

In [42]:
hub.push("thinker/summary_news", prompt)

'https://smith.langchain.com/hub/thinker/summary_news/2c3e83d5'

In [47]:
template = """
KEY를 가진 JSON 형식으로 답변해주세요.

KEY:
- text<입력받은 TEXT>
- summary<TEXT를 한 줄로 요약해주세요>
- translation<summary를 한국어로 번역해주세요>

TEXT: {text}
"""

In [48]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 기사를 요약하여 한국어로 번역한 후 정보를 전달하는 역할입니다."),
        ("human", template)
    ]
)
prompt

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 기사를 요약하여 한국어로 번역한 후 정보를 전달하는 역할입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='\nKEY를 가진 JSON 형식으로 답변해주세요.\n\nKEY:\n- text<입력받은 TEXT>\n- summary<TEXT를 한 줄로 요약해주세요>\n- translation<summary를 한국어로 번역해주세요>\n\nTEXT: {text}\n'))])

In [49]:
hub.push("thinker/summary_json_korean", prompt)

'https://smith.langchain.com/hub/thinker/summary_json_korean/9e3d6447'

In [50]:
def search_naver_news(keyword):
	# Make Request URL
	base_url = "https://openapi.naver.com/v1/search/news?"
	params = {
		"query": keyword,
		"display": 20,
		"sort": "sim"
	}
	headers = {
		"X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
		"X-Naver-Client-Secret": os.getenv("NAVER_SECRET_KEY")
	}

	url = base_url + urlencode(params)
	
	# Get response 
	request = Request(url, headers=headers)
	response = urlopen(request)

	# Get Text
	response_text = response.read().decode("utf-8")

	return response_text

output = search_naver_news("뉴진스")
print(type(output))
print(output)

<class 'str'>
{
	"lastBuildDate":"Tue, 04 Jun 2024 16:46:36 +0900",
	"total":90265,
	"start":1,
	"display":20,
	"items":[
		{
			"title":"<b>뉴진스<\/b> ‘하우 스위트’ 빌보드 글로벌 7위…통산 5번째 톱10",
			"originallink":"https:\/\/www.hani.co.kr\/arti\/culture\/culture_general\/1143354.html",
			"link":"https:\/\/n.news.naver.com\/mnews\/article\/028\/0002692216?sid=103",
			"description":"그룹 <b>뉴진스<\/b>의 신곡 ‘하우 스위트’가 빌보드 글로벌 차트 상위권에 들며 세계적인 인기를 증명했다. 미국 빌보드가 3일(현지시각) 공개한 최신 차트 결과를 보면, <b>뉴진스<\/b>의 ‘하우 스위트’는 글로벌(미국 제외)... ",
			"pubDate":"Tue, 04 Jun 2024 09:58:00 +0900"
		},
		{
			"title":"<b>뉴진스<\/b>, 대학축제 수익금 전액 어디 기부했나 봤더니…",
			"originallink":"https:\/\/www.hankyung.com\/article\/2024060321297",
			"link":"https:\/\/n.news.naver.com\/mnews\/article\/015\/0004992478?sid=102",
			"description":"그룹 <b>뉴진스<\/b>가 대학축제로 벌어들인 수익금 전액을 한국장학재단에 기부했다. 어도어는 3일 어도어와 <b>뉴진스<\/b> 이름으로 대학축제 수익금 전액을 한국장학재단에 기부했다고 밝혔다. 이 기부금은 저소득층 대학생의 생활비... ",
			"pubDate":"Mon, 03 Jun 2024 15:59:00 +0900"
		},
		{
			"title":"<b>

In [59]:
from langchain_openai import ChatOpenAI
question = "안녕"
model = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=True)
answer = model.invoke(question).content

In [86]:
import requests 
from bs4 import BeautifulSoup

url = "https://www.melon.com/chart/index.htm"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"}
test = requests.get(url, headers=headers)

soup = BeautifulSoup(test.text)
charts = soup.find_all('tr', class_="lst50")

In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Chrome WebDriver 생성
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 웹 페이지로 이동
driver.get("https://www.melon.com/chart/index.htm")

charts = driver.find_elements(By.CLASS_NAME, "lst50")



In [98]:
print(charts[0].find_element(By.CLASS_NAME, "rank").text)
print(charts[0].find_element(By.CLASS_NAME, "wrap_song_info").text)

1
Supernova
aespa


In [99]:
print(charts[0].find_element(By.CLASS_NAME, "cnt").text)

111,477


In [81]:
len(soup.select(".lst50:nth-child(5)"))

1

In [91]:
charts[5].find("span", class_="cnt").text


'\n총건수\r\n\t\t\t\t\t\t\t\t\t\t\t\t0\r\n\t\t\t\t\t\t\t\t\t\t\t'

In [96]:
charts[0].select("#lst50 > td:nth-child(8) > div > button")[0].text

'\n좋아요\n\n총건수\r\n\t\t\t\t\t\t\t\t\t\t\t\t0\r\n\t\t\t\t\t\t\t\t\t\t\t\n'

In [101]:
import streamlit as st 

tabs = st.tabs(["A", "B", "C"])
print(type(tabs))

<class 'tuple'>


In [106]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Chrome WebDriver 생성
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 웹 페이지로 이동
driver.get("https://www.naver.com")

# 검색어 입력 
input_box = driver.find_element(By.CSS_SELECTOR, "#query")
input_box.send_keys("Python")
driver.find_element(By.CSS_SELECTOR, "#search-btn").click()

# 로드 될 때까지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 창 닫기 
driver.quit()

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser

template = """\
# INSTRUCTION
- 당신은 리뷰를 정밀하게 분석하는 역할입니다. 
- REVIEW의 내용을 다음 KEYS에 맞춰 JSON 형식으로 출력해주세요.
- 각 key의 value는 content와 action으로 구분되어 있습니다.
- content는 key에 대한 평가 내용입니다. 해당되는 내용이 없다면 "-"으로 표현하세요.
- action은 content에 대한 반응입니다. "긍정", "중립", "부정"으로만 표현하세요.

# REVIEW: {review}

# KEYS
- favor
- employee
- environment
- etc
- menu
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o",streaming=True)
chain = prompt | model | JsonOutputParser()


In [13]:
review = "오랫만에 다녀왔습니다😋 반찬그릇이 바뀌었더라고요😊정갈해보였습니당~고기맛 여전히 좋구요!!!  직접구워주셔서 먹기만하면 되니까 좋아요❤️"

print(chain.invoke({"review": review}))


{'favor': {'content': '고기맛 여전히 좋구요', 'action': '긍정'}, 'employee': {'content': '직접구워주셔서 먹기만하면 되니까 좋아요', 'action': '긍정'}, 'environment': {'content': '반찬그릇이 바뀌었더라고요. 정갈해보였습니당', 'action': '긍정'}, 'etc': {'content': '오랫만에 다녀왔습니다', 'action': '중립'}, 'menu': {'content': '-', 'action': '-'}}


In [47]:
template = """\
# INSTRUCTION
- 당신은 레시피를 추천해주는 역할입니다.
- INGREDIENTS로 현재 사용자가 가지고 있는 음식 재료입니다.
- KEYS에 따라 JSON 형식에 맞춰 답변하세요.

# INGREDIENTS: {ingredients}

# KEYS:
- title: <요리명>
- difficulty: <'상/중/하'로 표현하세요>
- ingredients: <'재료(무게)' 리스트로 작성하세요`>
- calorie: <Kcal 단위로 작성하세요>
- recipe: <지속 시간과 행동을 상세히 번호리스트로 작성하세요.>
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o",streaming=True)
chain = prompt | model | JsonOutputParser()

result = chain.invoke(
    {"ingredients": ['계란', '소시지', '버섯']}
)
print(result)

{'title': '소시지 버섯 계란 볶음', 'difficulty': '하', 'ingredients': ['계란(2개)', '소시지(2개)', '버섯(100g)'], 'calorie': '350 Kcal', 'recipe': ['1. 계란을 풀어 준비합니다.', '2. 소시지와 버섯을 얇게 슬라이스합니다.', '3. 팬을 중불로 예열하고 약간의 기름을 두릅니다.', '4. 소시지를 팬에 넣고 3분 동안 볶습니다.', '5. 버섯을 추가하고 5분 동안 볶습니다.', '6. 준비한 계란을 팬에 부어 잘 섞이도록 볶습니다. (약 3분)', '7. 모든 재료가 잘 익으면 소금과 후추로 간을 맞춥니다.', '8. 완성된 소시지 버섯 계란 볶음을 그릇에 담아냅니다.']}


In [48]:
print(type(result))
print(result)

<class 'dict'>
{'title': '소시지 버섯 계란 볶음', 'difficulty': '하', 'ingredients': ['계란(2개)', '소시지(2개)', '버섯(100g)'], 'calorie': '350 Kcal', 'recipe': ['1. 계란을 풀어 준비합니다.', '2. 소시지와 버섯을 얇게 슬라이스합니다.', '3. 팬을 중불로 예열하고 약간의 기름을 두릅니다.', '4. 소시지를 팬에 넣고 3분 동안 볶습니다.', '5. 버섯을 추가하고 5분 동안 볶습니다.', '6. 준비한 계란을 팬에 부어 잘 섞이도록 볶습니다. (약 3분)', '7. 모든 재료가 잘 익으면 소금과 후추로 간을 맞춥니다.', '8. 완성된 소시지 버섯 계란 볶음을 그릇에 담아냅니다.']}


In [57]:
format = f"""- 요리명: {result['title']}
- 난이도: {result['difficulty']}
- 준비물: {', '.join(result['ingredients'])}
- 칼로리: {result['calorie']}
- 요리 순서:
{test}
"""

print(format)

- 요리명: 소시지 버섯 계란 볶음
- 난이도: 하
- 준비물: 계란(2개), 소시지(2개), 버섯(100g)
- 칼로리: 350 Kcal
- 요리 순서:
1. 계란을 풀어 준비합니다.
2. 소시지와 버섯을 얇게 슬라이스합니다.
3. 팬을 중불로 예열하고 약간의 기름을 두릅니다.
4. 소시지를 팬에 넣고 3분 동안 볶습니다.
5. 버섯을 추가하고 5분 동안 볶습니다.
6. 준비한 계란을 팬에 부어 잘 섞이도록 볶습니다. (약 3분)
7. 모든 재료가 잘 익으면 소금과 후추로 간을 맞춥니다.
8. 완성된 소시지 버섯 계란 볶음을 그릇에 담아냅니다.



In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import StrOutputParser

template = """\
# INSTRUCTION
- 당신은 레시피를 추천해주는 역할입니다.
- INGREDIENTS는 현재 사용자가 가지고 있는 음식 재료입니다.
- FORMAT에 맞춰 답변하세요.

# INGREDIENTS: {ingredients}

# FORMAT:
- 요리명: <볼드체로 표현하세요, 최종 소요시간을 괄호()로 표현해주세요>
- 난이도: <'상/중/하'로 표현하세요>
- 재료: <재료(무게)를 상세히 작성하세요>
- 칼로리:
- 방법: <지속 시간과 행동을 상세히 작성하세요>
1. 
2. 
...
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o",streaming=True)
chain = prompt | model | StrOutputParser()

result = chain.invoke(
    {"ingredients": ['계란', '소시지', '버섯']}
)
print(result)

- 요리명: **소시지 버섯 계란 볶음 (20분)**
- 난이도: 하
- 재료:
  - 계란 (3개)
  - 소시지 (100g)
  - 버섯 (150g)
  - 소금 (약간)
  - 후추 (약간)
  - 식용유 (1큰술)
- 칼로리: 400kcal (1인분 기준)

- 방법:
1. **재료 준비** (5분)
   - 계란을 깨서 볼에 넣고 소금과 후추로 간을 한 뒤 잘 섞어줍니다.
   - 소시지는 얇게 썰어줍니다.
   - 버섯은 깨끗이 씻어 슬라이스로 잘라줍니다.
2. **소시지와 버섯 볶기** (7분)
   - 팬에 식용유를 둘러 중간 불로 예열합니다.
   - 예열된 팬에 소시지를 넣고 약 3분간 볶아줍니다.
   - 소시지가 어느 정도 익으면 버섯을 넣고 함께 4분간 볶아줍니다.
3. **계란 추가** (5분)
   - 팬 한쪽에 공간을 만들어 계란물을 붓습니다.
   - 계란이 어느 정도 익어가면 팬 전체에 섞어가며 볶아줍니다.
4. **마무리** (3분)
   - 모든 재료가 잘 섞이고 익으면 마지막으로 소금과 후추로 간을 맞춥니다.
   - 접시에 담아내면 완성입니다.


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import StrOutputParser

template3 = """\
# INSTRUCTION
- 당신은 공공기관의 마케팅부서를 도와주는 역할입니다.
- TOPIC에서 주제를 추출한 후, 그에 대한 아이디어를 제시합니다.
- COLUMNS에 따라 표로 작성하세요.

# TOPIC: {topic}

# COLUMNS:
- NO <아이디어 번호> 
- Title <아이디어 주제, 20자 이내로 작성하세요>
- Description <아이디어 상세 내용, 500자 이내로 작성하세요>
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o",streaming=True)
chain = prompt | model | StrOutputParser()

result = chain.invoke(
    {"topic": "외국 관광객 유치"}
)
print(result)

| NO | Title                                  | Description                                                                                     |
|----|----------------------------------------|-------------------------------------------------------------------------------------------------|
| 1  | Cultural Festivals                     | Host annual cultural festivals showcasing traditional music, dance, food, and crafts to attract tourists. |
| 2  | Guided Historical Tours                | Develop guided tours of historical landmarks and heritage sites with multilingual guides.        |
| 3  | Culinary Experiences                   | Offer cooking classes and food tours to highlight local cuisine and culinary traditions.         |
| 4  | Nature and Adventure Packages          | Create adventure tourism packages featuring hiking, rafting, and eco-friendly activities.        |
| 5  | Social Media Campaigns                 | Launch targeted social media campaigns highlighting unique attr

In [11]:
import pandas as pd
from io import StringIO

# 마크다운 텍스트를 데이터프레임으로 변환
df = pd.read_csv(StringIO(result), sep="|", skipinitialspace=True, engine='python')

# 불필요한 첫 번째와 마지막 열 제거
df = df.iloc[:, 1:-1]

# 결과 확인
df

,NO,Title,Description
0,----,----------------------------------------,----------------------------------------------...
1,1,Cultural Festivals,Host annual cultural festivals showcasing trad...
2,2,Guided Historical Tours,Develop guided tours of historical landmarks a...
3,3,Culinary Experiences,Offer cooking classes and food tours to highli...
4,4,Nature and Adventure Packages,Create adventure tourism packages featuring hi...
5,5,Social Media Campaigns,Launch targeted social media campaigns highlig...
6,6,Influencer Collaborations,Partner with travel influencers to promote tou...
7,7,Local Art Exhibitions,Organize art exhibitions and workshops featuri...
8,8,Special Seasonal Events,"Introduce seasonal events, like cherry blossom..."
9,9,Interactive Museums,Develop interactive and immersive museum exhib...


In [14]:
import langchainhub
langchainhub

<module 'langchainhub' from 'c:\\Users\\user\\anaconda3\\envs\\langchain\\lib\\site-packages\\langchainhub\\__init__.py'>

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import StrOutputParser

template = """\
# INSTRUCTION
- 당신은 댓글의 긍부정 분류기입니다.
- SENTENCE의 긍부정을 판단하여 CATEGORY의 요소를 문자열로 출력하세요

# CATEGORY: [긍정, 부정]

# SENTENCE: {sentence}
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o",streaming=True)
chain = prompt | model

result = chain.invoke(
    {"sentence": "안보현..잘생기면 잘생기기만해..왜 스윗하기까지한거야..."}
)
print(result)

content='긍정' response_metadata={'finish_reason': 'stop'} id='run-238b5d30-fadf-4d82-9a9f-a9ac346fd958-0'


In [20]:
result = chain.invoke(
    {"sentence": "딘딘이 백패커에 없으니까 허전하다 다시와라"}
)
print(result)

content='부정' response_metadata={'finish_reason': 'stop'} id='run-e35e5a04-9afe-47bb-adb9-5ef77ffef130-0'


In [26]:
from konlpy.tag import Okt 

okt = Okt()

okt.pos("이겼네. 메이저리그 타자들이 무섭긴 무섭네.", norm=True)

[('이겼네', 'Verb'),
 ('.', 'Punctuation'),
 ('메이저리그', 'Noun'),
 ('타자', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('무섭긴', 'Adjective'),
 ('무섭네', 'Adjective'),
 ('.', 'Punctuation')]

In [27]:
okt.pos("이겼네. 메이저리그 타자들이 무섭긴 무섭네.", stem=True)

[('이기다', 'Verb'),
 ('.', 'Punctuation'),
 ('메이저리그', 'Noun'),
 ('타자', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('무섭다', 'Adjective'),
 ('무섭다', 'Adjective'),
 ('.', 'Punctuation')]

In [31]:
okt.pos("이대로만 가자.", stem=True)

[('이대', 'Modifier'), ('로만', 'Noun'), ('가다', 'Verb'), ('.', 'Punctuation')]

In [30]:
okt.pos("이대로만 가자.", norm=True)

[('이대', 'Modifier'), ('로만', 'Noun'), ('가자', 'Verb'), ('.', 'Punctuation')]